In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, Conv2D, GlobalAvgPool2D, Input
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
import numpy as np
from google.colab import drive


In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd drive/MyDrive/RiceLeafsDisease/

/content/drive/MyDrive/RiceLeafsDisease


In [6]:
def img_Data(dir_path,target_size,batch,class_lst,preprocessing):
  if preprocessing:
    gen_object = ImageDataGenerator(preprocessing_function=preprocessing)
  else:
    gen_object = ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path,target_size=target_size,
                                          batch_size=batch,class_mode='sparse',
                                   classes=class_lst, shuffle=True))

In [7]:
train_data_gen = img_Data("train",(224,224),150,os.listdir("train"),preprocess_input)
valid_data_gen = img_Data("test",(224,224),150,os.listdir("test"),preprocess_input)

Found 2105 images belonging to 6 classes.
Found 528 images belonging to 6 classes.


In [8]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(224,224,3),
    alpha=1.0, include_top=False, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000,
    classifier_activation='softmax')

9406464/9406464 [==============================] - 0s 0us/step


In [9]:
base_model.trainable = False

In [10]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
elst = callbacks.EarlyStopping(monitor= 'val_loss',patience=5,mode='min')
save_ck = callbacks.ModelCheckpoint('.mdl_wt.hdf5', save_best_only=True, monitor='val_loss',mode='min')

In [12]:
model.fit(train_data_gen,batch_size=150,validation_data=valid_data_gen,
          callbacks=[elst,save_ck],epochs=10)

Epoch 1/10
15/15 [==============================] - 537s 35s/step - loss: 0.8725 - accuracy: 0.6931 - val_loss: 4.4243 - val_accuracy: 0.2178
Epoch 2/10
15/15 [==============================] - 87s 6s/step - loss: 0.3035 - accuracy: 0.8912 - val_loss: 5.2574 - val_accuracy: 0.1932
Epoch 3/10
15/15 [==============================] - 83s 5s/step - loss: 0.1963 - accuracy: 0.9325 - val_loss: 5.8286 - val_accuracy: 0.1837
Epoch 4/10
15/15 [==============================] - 83s 5s/step - loss: 0.1524 - accuracy: 0.9411 - val_loss: 6.0622 - val_accuracy: 0.2254
Epoch 5/10
15/15 [==============================] - 83s 6s/step - loss: 0.1252 - accuracy: 0.9601 - val_loss: 6.2588 - val_accuracy: 0.1913
Epoch 6/10
15/15 [==============================] - 83s 6s/step - loss: 0.0895 - accuracy: 0.9672 - val_loss: 6.5956 - val_accuracy: 0.1932
